## "MAANG Mosaic Strategy"

### <u>Problem Statement:</u>

**Background:**
MAANG stocks represent some of the most influential and dynamic companies within the tech sector, often showing significant volatility and trading volume. The performance of these stocks is keenly watched by investors for signs of broader market trends and potential investment opportunities. 

**Problem:**
There is a need to develop effective trading strategies tailored for a portfolio of MAANG stocks. Moreover, there is a need for systematic backtesting to validate these strategies' robustness before we move on the execute or forward testing.

**Objective:**
The goal is to implement a portfolio of diverse, sophisticated trading strategies tailored to the unique characteristics of each MAANG stock. This comprehensive approach aims to optimize returns and manage risks by leveraging machine learning, data analysis, and quantitative models to predict stock movements and identify trading opportunities

### Asset Allocation:

Based on the target time frame, we will use the Scipy Optimizer and Monte Carlo Simulation to find the weights for the GMV portfolio based on expected return and volatility. This will guide us in allocating our initial capital for different stocks.

In [1]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import math
from tqdm.notebook import tqdm
import yfinance as yf
import matplotlib.pyplot as plt
import scipy.optimize as opt
import warnings 
warnings.filterwarnings("ignore")
from loguru import logger
from typing import Union
from colorama import Fore, Style

In [2]:
# Create a class to fetch and store the stock data from Yfinance

class DataExtractor:
    def __init__(self, ticker_list, start_date, end_date, interval='1d'):
        self.ticker_list = ticker_list
        self.start_date = start_date
        self.end_date = end_date
        self.stock_data = self.get_stock_data()
        
    def get_stock_data(self):
        stock_data = yf.download(self.ticker_list, start=self.start_date, end=self.end_date)
        return stock_data

In [3]:
MAANG = ["META","AAPL","AMZN","NFLX","GOOGL"]
extractor = DataExtractor(ticker_list=MAANG, start_date='2015-01-01', end_date='2020-01-01')

[*********************100%%**********************]  5 of 5 completed


In [4]:
# Process the multi-index dataframe to only contain closing price

d = {}
for ticker in MAANG:
    d[ticker] = extractor.stock_data[('Close',ticker)].values.tolist()
    
price_df = pd.DataFrame(d)

In [5]:
# create a class function to calculate portfolio statistics
class PortfolioStatistics:
    def __init__(self, price_df, weights):
        self.price_df = price_df
        self.weights = weights
        self.daily_return = self.calculate_daily_return()
        self.portfolio_return = self.calculate_portfolio_return()
        self.portfolio_volatility = self.calculate_portfolio_volatility()
        self.portfolio_sharpe_ratio = self.calculate_sharpe_ratio()
        
    def calculate_daily_return(self):
        daily_return = self.price_df.pct_change()
        return daily_return
    
    def calculate_portfolio_return(self):
        portfolio_return = self.daily_return.mean() * 252
        return portfolio_return
    
    def calculate_portfolio_volatility(self):
        portfolio_volatility = self.daily_return.cov() * 252
        return portfolio_volatility
    
    def calculate_sharpe_ratio(self):
        sharpe_ratio = self.portfolio_return / self.portfolio_volatility
        return sharpe_ratio
    
    def get_stats(self):
        weights = np.array(self.weights)
        portfolio_return = np.sum(self.daily_return.mean() * weights) * 252
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(self.daily_return.cov() * 252, weights)))
        sharpe_ratio = portfolio_return / portfolio_volatility
        return portfolio_return, portfolio_volatility, sharpe_ratio
    

In [6]:
# set seed
np.random.seed(42)

num_iter = 100000

portfolio_ret_list = []
portfolio_vol_list = []
w_list =[]

max_sharpe = 0
max_sharpe_var = None
max_sharpe_ret = None
max_sharpe_w = None

for i in tqdm(range(num_iter)):
    weights = np.random.random(len(MAANG))
    weights /= np.sum(weights)
    portfolio = PortfolioStatistics(price_df, weights)
    portfolio_return, portfolio_volatility,sharpe_ratio = portfolio.get_stats()
    
    if sharpe_ratio > max_sharpe:
        max_sharpe = sharpe_ratio
        max_sharpe_vol= portfolio_volatility
        max_sharpe_ret = portfolio_return
        max_sharpe_w = weights

    portfolio_vol_list.append(portfolio_volatility)
    portfolio_ret_list.append(portfolio_return)
    w_list.append(weights)
    
logger.info(f"{Fore.GREEN}Portfolio with maximum Sharpe Ratio: {max_sharpe}\n"
            f"Return: {max_sharpe_ret}\n"
            f"Volatility: {max_sharpe_vol}\n"
            f"Weights: {max_sharpe_w}{Style.RESET_ALL}")

# logger.info(f"{Fore.BLUE}Portfolio with minimum Volatility: {min(portfolio_vol_list)}\n"
#             f"Return: {portfolio_ret_list[portfolio_vol_list.index(min(portfolio_vol_list))]}\n"
#             f"Volatility: {min(portfolio_vol_list)}\n"
#             f"Weights: {w_list[portfolio_vol_list.index(min(portfolio_vol_list))]}{Style.RESET_ALL}")

  0%|          | 0/100000 [00:00<?, ?it/s]

2024-03-13 00:35:33.706 | INFO     | __main__:<module>:31 - Portfolio with maximum Sharpe Ratio: 1.4714884909578305
Return: 0.38866506657386285
Volatility: 0.26413055145329106
Weights: [0.03223165 0.11486156 0.63887445 0.21321606 0.00081628]


##### Remark:
Assume we begin with $100000.

We will assign the initial capital based on the weights generated from the Monte Carlo Simulation which maximizes the Sharpe

In [7]:
for i in range(len(MAANG)):
    print(f"{MAANG[i]}: {max_sharpe_w[i]*100:.2f}%, ${max_sharpe_w[i]*100000:.2f}")

META: 3.22%, $3223.16
AAPL: 11.49%, $11486.16
AMZN: 63.89%, $63887.45
NFLX: 21.32%, $21321.61
GOOGL: 0.08%, $81.63


#### Strategy for META: SuperTrend

The SuperTrend trading strategy is a trend-following approach that uses the SuperTrend indicator, combining price movement and volatility to determine the direction of the trend and generate buy or sell signals. The SuperTrend indicator is calculated based on the Average True Range (ATR) and a multiplier to define the trend. A buy signal is generated when the price closes above the SuperTrend line, indicating a bullish trend, and a sell signal is generated when the price closes below the SuperTrend line, indicating a bearish trend. Traders often use these signals to open or close positions, with the aim of capturing the momentum of the market trend while managing risk through stop-loss orders set around the SuperTrend line.

In [3]:
meta_extractor = DataExtractor(ticker_list="META", start_date='2015-01-01', end_date='2020-01-01')
meta_df = meta_extractor.stock_data.reset_index()

[*********************100%%**********************]  1 of 1 completed


In [4]:
meta_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-01-02,78.580002,78.930000,77.699997,78.449997,78.366852,18177500
1,2015-01-05,77.980003,79.250000,76.860001,77.190002,77.108192,26452200
2,2015-01-06,77.230003,77.589996,75.360001,76.150002,76.069290,27399300
3,2015-01-07,76.760002,77.360001,75.820000,76.150002,76.069290,22045300
4,2015-01-08,76.739998,78.230003,76.080002,78.180000,78.097137,23961000


In [ ]:
class SuperTrendVectorBacktester:
    def __init__(
        self,
        df: pd.DataFrame,
        initial_capital: Union[int, float],
        in_position: bool = False,
        period = 10,
        multiplier = 3,
    ):
        self.df = df
        self.initial_capital = initial_capital
        self.in_position = in_position
        self.period = period
        self.multiplier = multiplier
        self.balance = self.initial_capital
        self.no_of_shares = 0
        
    def calculate_atr(self, df):
        df['High-Low'] = df['High'] - df['Low']
        df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
        df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
        df['TR'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
        df['ATR'] = df['TR'].rolling(self.period).mean()
        return df
    
    def calculate_supertrend(self, df):
        self.calculate_atr(df)
        hl2 = (df['High'] + df['Low']) / 2
        df['Final Upperband'] = hl2 + (self.multiplier * df['ATR'])
        df['Final Lowerband'] = hl2 - (self.multiplier * df['ATR'])
        df['Supertrend'] = np.nan

        for current in range(1, len(df)):
            previous = current - 1
            if df['Close'].iloc[current] > df['Final Upperband'].iloc[previous]:
                df.loc[current, 'Supertrend'] = df['Final Lowerband'].iloc[current]
            elif df['Close'].iloc[current] < df['Final Lowerband'].iloc[previous]:
                df.loc[current, 'Supertrend'] = df['Final Upperband'].iloc[current]
            else:
                df.loc[current, 'Supertrend'] = df['Supertrend'].iloc[previous]
                if df['Supertrend'].iloc[current] == df['Final Upperband'].iloc[previous] and df['Close'].iloc[current] <= df['Final Upperband'].iloc[current]:
                    df.loc[current, 'Supertrend'] = df['Final Lowerband'].iloc[current]
                elif df['Supertrend'].iloc[current] == df['Final Lowerband'].iloc[previous] and df['Close'].iloc[current] >= df['Final Lowerband'].iloc[current]:
                    df.loc[current, 'Supertrend'] = df['Final Upperband'].iloc[current]
        return df
    
    def calculate_daily_returns(self, df):
        df["Market Returns"] = df["Close"].pct_change()
        return df
    
    def create_additional_columns(self, df):
        initial_values = {
            "In Position": (self.in_position, "bool"),
            "Balance": (self.balance, "float64"),
            "No of Shares": (self.no_of_shares, "float64"),
        }

        for column, (initial_value, dtype) in initial_values.items():
            df[column] = pd.Series(dtype=dtype)
            df.loc[0, column] = initial_value

        return df
    
    def buy_signal(self, df, i):
        if df["Close"][i] > df["Supertrend"][i] and not self.in_position: 
            return True
    
    def sell_signal(self, df, i):
        if df["Supertrend"][i] > df["Close"][i] and self.in_position:
            return True
        
    def backtest_strategy(self, df):
        for i in range(1, len(df)):
            close_price = df.loc[i, "Close"]
            if self.buy_signal(df, i):
                self.no_of_shares = math.floor(self.balance / close_price)
                self.balance -= self.no_of_shares * close_price
                self.in_position = True
            elif self.sell_signal(df, i):
                self.balance += self.no_of_shares * close_price
                self.no_of_shares = 0
                self.in_position = False

            df.loc[i, ["No of Shares", "Balance", "In Position"]] = self.no_of_shares, self.balance, self.in_position

        if self.in_position:
            self.balance += self.no_of_shares * df.loc[df.index[-1], "Close"]
            self.no_of_shares = 0
            self.in_position = False
            df.loc[df.index[-1], ["No of Shares", "Balance", "In Position"]] = self.no_of_shares, self.balance, self.in_position

        return df
    
    def create_signal_column(self, df):
        df["In Position Shift"] = df["In Position"].shift(1)
        df.loc[
            (df["In Position"] == True) & (df["In Position Shift"] == False), "Signal"
        ] = "Buy"
        df.loc[
            (df["In Position"] == False) & (df["In Position Shift"] == True), "Signal"
        ] = "Sell"
        df["Signal"].fillna("Hold", inplace=True)
        return df

    def calculate_strategy_capital_change(self, df):
        df["Total Strategy Capital"] = df["Balance"] + df["No of Shares"] * df["Close"]
        return df

    def calculate_buy_and_hold_capital_change(self, df):
        left_over = (
            self.initial_capital
            - math.floor(self.initial_capital / df.loc[0, "Close"]) * df.loc[0, "Close"]
        )
        df["Total Buy and Hold Capital"] = (
            self.initial_capital * (1 + df["Market Returns"]).cumprod() + left_over
        )
        df["Total Buy and Hold Capital"].fillna(self.initial_capital, inplace=True)
        return df
    
    def calculate_realized_pnl(self, df):
        df_subset = df.copy().query("Signal!='Hold'")
        df_subset["Signal Shift"] = df_subset["Signal"].shift(1)
        df_subset.loc[
            (df_subset["Signal"] == "Sell") & (df_subset["Signal Shift"] == "Buy"),
            "Realized PnL",
        ] = df_subset["Total Strategy Capital"] - df_subset[
            "Total Strategy Capital"
        ].shift(
            1
        )
        df = df.merge(df_subset[["Date", "Realized PnL"]], on="Date", how="left")
        return df

    def calculate_unrealized_pnl(self, df):
        df_subset_2 = df.copy().query("Signal!='Sell'")

        reference = 0
        for index, row in df_subset_2.iterrows():
            if row["Signal"] == "Buy":
                reference = row["Total Strategy Capital"]
            elif row["Signal"] == "Hold" and row["No of Shares"] > 0:
                df_subset_2.loc[index, "Unrealized PnL"] = (
                    row["Total Strategy Capital"] - reference
                )

        df = df.merge(df_subset_2[["Date", "Unrealized PnL"]], on="Date", how="left")
        return df

    def calculate_strategy_drawdown_and_max_drawdown(self, df):
        # Calculate the running maximum
        df["Strategy Cumulative Max"] = df["Total Strategy Capital"].cummax()
        # Calculate the drawdown
        df["Strategy Drawdown"] = (
            df["Total Strategy Capital"] - df["Strategy Cumulative Max"]
        ) / df["Strategy Cumulative Max"]
        # Calculate the max drawdown
        df["Strategy Max Drawdown"] = df["Strategy Drawdown"].min()
        return df

    def calculate_buy_and_hold_drawdown_and_max_drawdown(self, df):
        # Calculate the running maximum
        df["Buy and Hold Cumulative Max"] = df["Total Buy and Hold Capital"].cummax()
        # Calculate the drawdown
        df["Buy and Hold Drawdown"] = (
            df["Total Buy and Hold Capital"] - df["Buy and Hold Cumulative Max"]
        ) / df["Buy and Hold Cumulative Max"]
        # Calculate the max drawdown
        df["Buy and Hold Max Drawdown"] = df["Buy and Hold Drawdown"].min()
        return df
    
    def reorganize_columns(self, df):
        columns = [
            "Date",
            "Close",
            "Supertrend",
            "Market Returns",
            "Signal",
            "In Position",
            "No of Shares",
            "Balance",
            "Realized PnL",
            "Unrealized PnL",
            "Total Strategy Capital",
            "Total Buy and Hold Capital",
            "Strategy Drawdown",
            "Strategy Max Drawdown",
            "Buy and Hold Drawdown",
            "Buy and Hold Max Drawdown",
        ]
        df = df[columns]
        df["Date"] = pd.to_datetime(df["Date"]).dt.date
        return df
    
    def backtesting_flow(self):
        df = self.df.copy().reset_index()
        df = self.calculate_supertrend(df)
        df = self.calculate_daily_returns(df)
        df = self.create_additional_columns(df)
        df = self.backtest_strategy(df)
        df = self.create_signal_column(df)
        df = self.calculate_strategy_capital_change(df)
        df = self.calculate_buy_and_hold_capital_change(df)
        df = self.calculate_realized_pnl(df)
        df = self.calculate_unrealized_pnl(df)
        df = self.calculate_strategy_drawdown_and_max_drawdown(df)
        df = self.calculate_buy_and_hold_drawdown_and_max_drawdown(df)
        df = self.reorganize_columns(df)
        return df
        
    
    
    
    

In [ ]:
test = SuperTrendVectorBacktester(meta_df, 10000, period=8, multiplier=2)

In [ ]:
test.backtesting_flow()